In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''

sdata = read_sql_query(query,engine,parse_dates=True)
sdata.id_patient = sdata.id_patient.astype(Int32Dtype())

# close the pool of connection
engine.dispose()

True

In [3]:
les3CAP =  DataFrame({
    "case_id":[
        "b06a13e9-2990-409c-82b4-387448dd2403",
        "637ac619-cc73-411f-b936-c2d1ecf19a78",
        "234dce01-e600-47b2-b940-ac4ec64cb71a"
    ]
})

les3CAP.case_id.count()
sdata.case_id.count()

3

68868

In [4]:
les3CAP_case_id = sdata[sdata.case_id.isin(les3CAP.case_id)]
les3CAP_case_id

,id_patient,case_id,code,first_name,last_name,age,dob,interview_date,e__telefn,adress,...,interviewer_lastname,interviewer_role,interviewer_other_info,interview_location,is_your_parent_alive,mothers_name,fathers_name,who_is_your_law_parent,total,organisation
9345,75433,234dce01-e600-47b2-b940-ac4ec64cb71a,CAP/DRMS/100071408,Roseline,Pierre Gilles,18.0,2003-06-02,2020-07-25,35520841,Fort St Michel rue Boukman,...,Bycinthe,mentor,None,Fort St Michel,0.0,,Julius Jean Pierre,,35.0,CARIS
26784,90067,637ac619-cc73-411f-b936-c2d1ecf19a78,CAP/DRMS/098184208,Rose-Martine,François,12.0,2009-08-30,2020-08-03,49419582,Bas-Aviation,...,Bycinthe,mentor,None,Fort St Michel,2.0,Chery Manouchka,,,18.0,CARIS
47603,75398,b06a13e9-2990-409c-82b4-387448dd2403,CAP/DRMS/100071309,Gudailove,Altenor,17.0,2005-05-24,2020-05-29,48305308,laury,...,Inelus,travay_sosyal_sikolg,None,LagosetteLaury,2.0,Altenar Siliana,,Altena Siliana,23.0,CARIS


In [5]:
les3CAP_case_id.to_excel('./kap_CAP_not_found.xlsx',na_rep="",index=False)